In [214]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

k = 4 # we know it's 4
alpha = 1
eta = 0.001

categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']
twenty_train = fetch_20newsgroups(
  subset='train', 
  categories=categories,
  shuffle=True,
  random_state=42)

document_set = twenty_train.data[0:500]

In [215]:
from nltk.tag import pos_tag
import re

edited_set = []
for document in document_set:
    document_no_numbers = re.sub("\d+", " ", document)
    tagged_document = pos_tag(document_no_numbers.split())
    edited_document = [word for word, tag in tagged_document if tag != 'NNP' and tag != 'NNPS']
    edited_set.append(' '.join(edited_document))

In [216]:
count_vect = CountVectorizer(stop_words='english')
bag_of_words = count_vect.fit_transform(edited_set) # D x W
vocabulary = count_vect.vocabulary_ # W words to indices

Generate list of lists of vocabulary indices for tokens in each document

In [217]:
tokenizer = count_vect.build_tokenizer()

# generate list of lists of vocabulary indices for tokens in each document
document_vocabulary_indices = []
for document in edited_set:
    vocabulary_indices_for_document = []
    tokens = tokenizer(document)
    for token in tokens:
        if token.lower() in vocabulary.keys():
            vocabulary_indices_for_document.append(vocabulary[token.lower()])
    document_vocabulary_indices.append(vocabulary_indices_for_document)

Generate `topic_assignments`: A D-length array with variable-length inner arrays. Each inner array is a list of integers representing the token's topic assignment.

In [218]:
topic_assignments = []
for document in document_vocabulary_indices:
    topic_assignments_for_document = []
    for token in document:
        draws = np.random.multinomial(1, [1/k]*k)
        assignment = np.argwhere(draws==1)[0][0]
        topic_assignments_for_document.append(assignment)
    topic_assignments.append(topic_assignments_for_document)

Generate `document_topic_counts`: A D x K matrix having counts of tokens assigned to topic k for document d.

In [219]:
document_topic_counts = np.zeros((len(edited_set), k))

# for every d document count number of times k shows up
for d_index in range(document_topic_counts.shape[0]):
    document_topic_assignments = topic_assignments[d_index]
    for k_index in range(document_topic_counts.shape[1]):
        topic_count = sum(1 for i in topic_assignments[d_index] if i == k_index)
        document_topic_counts[d_index, k_index] = topic_count

Generate `word_topic_counts`: A W x K matrix having count that token w appears in topic k.

In [220]:
word_topic_counts = np.zeros((len(vocabulary.keys()), k))

# for every word's topic assignment, find the token's vocabulary index
# and increment `word_topic_counts[token_index, k]`
for doc_index, document in enumerate(topic_assignments):
    for word_document_index, word_topic_assignment in enumerate(document):
        word_vocabulary_index = document_vocabulary_indices[doc_index][word_document_index]
        word_topic_counts[word_vocabulary_index, word_topic_assignment] += 1

In [223]:
# dumb sanity check: for every iteration, the average probability should be increasing
iter_probs_averages = []

for i in range(10):
    iter_probs = []
    for doc_index, d in enumerate(document_vocabulary_indices):
        for word_document_index, word_vocab_index in enumerate(d):
            current_topic_assignment = topic_assignments[doc_index][word_document_index]
            document_topic_counts[doc_index, current_topic_assignment] -= 1
            word_topic_counts[word_vocab_index, current_topic_assignment] -= 1
            document_token_count = len(d) + k*alpha
            # p(topic t | document d): proportion of words in document d that are assigned to topic t
            b = (document_topic_counts[doc_index,])/document_token_count
            # p(word w| topic t): proportion of assignments to topic t, over all documents d, that come from word w
            topic_tokens_count = np.sum(word_topic_counts, 0)[current_topic_assignment]
            a = (word_topic_counts[word_vocab_index,] + eta) / topic_tokens_count
            p_z = b*a
            new_probs = p_z/np.sum(p_z)
            iter_probs.append(np.max(new_probs))
            draws = np.random.multinomial(1, new_probs)
            new_assignment = np.argwhere(draws==1)[0][0]
            topic_assignments[doc_index][word_document_index] = new_assignment
            document_topic_counts[doc_index, new_assignment] += 1
            word_topic_counts[word_vocab_index, new_assignment] += 1
    iter_probs_averages.append(np.average(iter_probs))

iter_probs_averages

[0.94928108890864882,
 0.95388177838349453,
 0.95611225420436585,
 0.95719499977061673,
 0.95826723988781159,
 0.95875602219687095,
 0.95902547514859338,
 0.95919885743300881,
 0.95948200110887616,
 0.95973040512642682]

In [222]:
inv_vocab = {v: k for k, v in vocabulary.items()}

# number of times word appears in topic k / number of words in topic k
number_of_words_in_topics = np.sum(word_topic_counts, 0)
phi = np.divide(word_topic_counts, number_of_words_in_topics)

# iterate through k topics and find words which have highest probability
for i in range(k):
    vocab_topic = np.argsort(-phi[:,i])[0:10]
    for vocab_idx in vocab_topic:
        print(inv_vocab[vocab_idx])
    print('')

standard
absolute
meaning
effects
necessary
heard
levels
universe
including
deal

pain
motto
various
versions
water
surface
users
choose
treat
bacteria

edu
writes
people
com
don
article
think
like
know
does

edu
graphics
group
help
geb
long
version
cause
idea
mail

